In [ ]:
import pandas as pd 
import re
import string

df =pd.read_csv("birlesmis_dosya.csv")
df=df.iloc[:,:2]
df.head()



In [ ]:



def temizle_metin(metin):
    
    if not isinstance(metin, str):
        metin = str(metin)
    
    
    metin = metin.lower()

    
    metin = re.sub(r'[^\x00-\x7F]+', ' ', metin)

  
    metin = metin.translate(str.maketrans('', '', string.punctuation))

    
    metin = re.sub(r'\s+', ' ', metin).strip()

    return metin


In [ ]:
df["yorum"].apply(lambda x: type(x)).value_counts()

In [ ]:
df = df.dropna(subset=["yorum"])  

df[~df["yorum"].apply(lambda x: isinstance(x, str))]

df["yorum"] = df["yorum"].apply(temizle_metin)

print(df)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tVec=TfidfVectorizer()

X_tfidf = tVec.fit_transform(df["yorum"])

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tVec.get_feature_names_out())
                        
df_birlesik = pd.concat([df, df_tfidf], axis=1)

df_birlesik.to_csv("testValue.csv", index=False)

X=df_birlesik.iloc[:,2:]
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

y = df["puan"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(y)

In [ ]:

model = LogisticRegression()
model.fit(X_train, y_train)

# Tahmin yap
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))



In [ ]:
print(y_test,y_pred)



In [ ]:
yeni_cumle = "alın"


temiz = temizle_metin(yeni_cumle)


yeni_vektor = tVec.transform([temiz])

tahmin = model.predict(yeni_vektor)

print("Tahmin edilen sınıf:", tahmin[0])